In [4]:
import pandas as pd

# Load dataset from the "archive" folder
data = pd.read_csv("archive/kl.csv", encoding="latin1")

# Show first 5 rows
data.head()


,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31.0,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94.0,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226.5M
1,1,20801,Cristiano Ronaldo,33.0,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94.0,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127.1M
2,2,190871,Neymar Jr,26.0,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92.0,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228.1M
3,3,193080,De Gea,27.0,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91.0,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138.6M
4,4,192985,K. De Bruyne,27.0,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91.0,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196.4M


In [6]:
# --- FIFA 19 dataset cleaning script (ready to run) ---

import re
import numpy as np
import pandas as pd

# 1) Load (this dataset is usually latin1-encoded)
df = pd.read_csv("archive/kl.csv", encoding="latin1", low_memory=False)

# 2) Drop obvious junk/URL columns & unnamed index columns
to_drop = [c for c in df.columns if c.lower().startswith("unnamed")]
to_drop += [c for c in ["Photo","Flag","Club Logo"] if c in df.columns]
df = df.drop(columns=to_drop, errors="ignore")

# 3) Tidy column names a bit (optional: keep human-friendly)
#    Here we only strip stray spaces
df.columns = df.columns.str.strip()

# 4) De-duplicate: keep the highest Overall per player ID if duplicates exist
if "ID" in df.columns and "Overall" in df.columns:
    df = (df.sort_values(["ID","Overall"], ascending=[True,False])
            .drop_duplicates(subset=["ID"], keep="first"))

# ---------- helpers ----------
def parse_euro(x):
    """Convert strings like '€110M', '€500K', '€0' -> numeric euros (float).
       Returns NaN if not parseable."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip().replace("€","").replace(",","")
    if s == "" or s.lower() == "none":
        return np.nan
    mult = 1.0
    if s.endswith(("M","m")):
        mult = 1e6
        s = s[:-1]
    elif s.endswith(("K","k")):
        mult = 1e3
        s = s[:-1]
    try:
        return float(s) * mult
    except:
        return np.nan

def parse_height(h):
    """Convert heights like 5'11" to centimeters.
       If already numeric-like, try to parse as cm."""
    if pd.isna(h):
        return np.nan
    s = str(h).strip()
    # Common FIFA format: 5'11
    m = re.match(r"^\s*(\d+)\s*'\s*(\d+)\s*\"?\s*$", s)
    if m:
        feet = int(m.group(1)); inches = int(m.group(2))
        cm = (feet*12 + inches) * 2.54
        return round(cm, 1)
    # Sometimes just inches like 71
    if re.fullmatch(r"\d+(\.\d+)?", s):
        # assume cm if plausible
        v = float(s)
        return v if v > 60 else round(v*2.54,1)  # if suspiciously small, treat as inches
    return np.nan

def parse_weight(w):
    """Convert '176lbs' to kilograms."""
    if pd.isna(w):
        return np.nan
    s = str(w).lower().replace(" ", "")
    m = re.match(r"^(\d+)(lb|lbs)$", s)
    if m:
        lb = float(m.group(1))
        return round(lb * 0.45359237, 1)
    # numeric -> assume kg if reasonable
    if re.fullmatch(r"\d+(\.\d+)?", s):
        v = float(s)
        return v if v > 30 else np.nan
    return np.nan

def extract_rating(v):
    """Turn strings like '87+2' or '94' into numeric 87 or 94."""
    if pd.isna(v):
        return np.nan
    m = re.search(r"(\d+)", str(v))
    return float(m.group(1)) if m else np.nan
# -----------------------------

# 5) Convert money-like columns
for col in ["Value","Wage","Release Clause"]:
    if col in df.columns:
        df[col] = df[col].apply(parse_euro)

# 6) Convert Height/Weight
if "Height" in df.columns:
    df["Height_cm"] = df["Height"].apply(parse_height)
if "Weight" in df.columns:
    df["Weight_kg"] = df["Weight"].apply(parse_weight)

# 7) Split 'Work Rate' into two columns like 'High/Medium'
if "Work Rate" in df.columns:
    wr = df["Work Rate"].astype(str).str.split("/", n=1, expand=True)
    if wr.shape[1] == 2:
        df["WorkRate_Attack"] = wr[0].str.strip()
        df["WorkRate_Defense"] = wr[1].str.strip()

# 8) Parse dates / years
if "Joined" in df.columns:
    df["Joined_dt"] = pd.to_datetime(df["Joined"], errors="coerce")
if "Contract Valid Until" in df.columns:
    # Sometimes it's a year; coerce to numeric year
    df["Contract Valid Until"] = pd.to_numeric(df["Contract Valid Until"], errors="coerce")

# 9) Make GK/outfield handling easier (optional)
# Extract numeric from the positional overall columns (LS, ST, RW, LB, ...), which often look like '87+2'
pos_cols = [
    "LS","ST","RS","LW","LF","CF","RF","RW",
    "LAM","CAM","RAM","LM","LCM","CM","RCM","RM",
    "LWB","LDM","CDM","RDM","RWB","LB","LCB","CB","RCB","RB"
]
for c in pos_cols:
    if c in df.columns:
        df[c] = df[c].apply(extract_rating)

# GK attributes are already numeric; leave them as-is but they’ll be NaN for outfield players.

# 10) Handle missing values (simple, safe defaults)
#    - Categorical: fill with 'Unknown'
#    - Numeric: fill with median (per column)
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

df[cat_cols] = df[cat_cols].fillna("Unknown")
for c in num_cols:
    df[c] = df[c].fillna(df[c].median())

# 11) Final small tidy: keep either original or engineered versions
#     (Optional) Drop original Height/Weight if you only want cm/kg versions
# df = df.drop(columns=["Height","Weight"], errors="ignore")

# 12) Save cleaned data
df.to_csv("archive/kl_clean.csv", index=False, encoding="utf-8")

# 13) Quick sanity report
print("Rows, Cols:", df.shape)
print("\nSample:")
display(df.head(3))
print("\n% Missing by column (after fills, should be 0 for most):")
print((df.isna().sum()/len(df)*100).round(2).sort_values(ascending=False).head(10))
print("\nSaved -> archive/kl_clean.csv")


Rows, Cols: (18207, 90)

Sample:


,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,...,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Height_cm,Weight_kg,WorkRate_Attack,WorkRate_Defense,Joined_dt
4014,16,Luis García,37.0,Spain,71.0,71,KAS Eupen,NaN,NaN,1906,...,12.0,13.0,11.0,11.0,NaN,177.8,64.9,Medium,Medium,2014-07-19
76,41,Iniesta,34.0,Spain,86.0,86,Vissel Kobe,NaN,NaN,2058,...,13.0,6.0,13.0,7.0,NaN,170.2,68.0,High,Medium,2018-07-16
725,80,E. Belözo?lu,37.0,Turkey,79.0,79,Medipol Ba?ak?ehir FK,NaN,NaN,2047,...,11.0,7.0,14.0,8.0,NaN,170.2,72.1,Medium,Medium,2015-07-09



% Missing by column (after fills, should be 0 for most):
Value             100.00
Wage              100.00
Release Clause    100.00
Joined_dt           8.53
Name                0.00
ID                  0.00
Potential           0.00
Overall             0.00
Age                 0.00
Nationality         0.00
dtype: float64

Saved -> archive/kl_clean.csv


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load your cleaned dataset
data = pd.read_csv("archive/kl_clean.csv")

# Select features (X) and target (y)
features = ["Age", "Potential", "Crossing", "Finishing", "Dribbling", 
            "LongPassing", "BallControl", "Stamina", "Strength", "Vision"]
X = data[features]
y = data["Overall"]

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R² Score:", r2)

# Show first 10 predictions vs actual
results = pd.DataFrame({"Actual": y_test[:10].values, "Predicted": y_pred[:10]})
print(results)


Mean Squared Error: 5.910690103923701
R² Score: 0.8737852453306516
   Actual  Predicted
0    59.0  58.087335
1    63.0  62.517361
2    73.0  71.842656
3    72.0  71.678242
4    68.0  67.225607
5    72.0  70.659380
6    69.0  67.647231
7    70.0  67.618740
8    77.0  74.111567
9    64.0  65.522677
